In [1]:
def create_dummy_input_file(filename, F):
    """Create an input file for testing. Does not use unlimited
    dimensions, creates one time record only."""

    import netCDF4

    nc = netCDF4.Dataset(filename, "w")

    Mx = 88
    My = 152
    Mz = 11
    for name, length in [["x", Mx], ["y", My], ["z", Mz], ["time", 1]]:
        nc.createDimension(name, length)
        nc.createVariable(name, "f8", (name,))

    # use X and Y ranges corresponding to a grid covering Greenland
    x = np.linspace(-669650.0, 896350.0, Mx)
    y = np.linspace(-3362600.0, -644600.0, My)
    z = np.linspace(0, 4000.0, Mz)
    # the single time record
    time = [0.0]

    for name, data in [["x", x], ["y", y], ["z", z], ["time", time]]:
        nc.variables[name][:] = data

    nc.proj4 = "epsg:3413"

    xx, yy = np.meshgrid(x, y)

    def write(prefix, dimensions):
        "Write test data to the file using given storage order."
        name = prefix + "_".join(dimensions)

        slices = {"x": slice(0, Mx), "y": slice(0, My), "time": 0, "z": None}

        if "z" in dimensions:
            # set fill_value and coordinates in variables with the z
            # dimensions and not others (just so that we can get
            # better test coverage)
            variable = nc.createVariable(name, "f8", dimensions, fill_value=-2e9)
            variable.coordinates = "lon lat"
        else:
            variable = nc.createVariable(name, "f8", dimensions)

        variable.long_name = name + " (let's make it long!)"

        # set indexes for all dimensions (z index will be re-set below)
        indexes = [Ellipsis] * len(dimensions)
        for k, d in enumerate(dimensions):
            indexes[k] = slices[d]

        # transpose 2D array if needed
        if dimensions.index("y") < dimensions.index("x"):

            def T(x):
                return x

        else:
            T = np.transpose

        if "z" in dimensions:
            for k in range(Mz):
                indexes[dimensions.index("z")] = k
                variable[indexes] = T(F(xx, yy, z[k]))
        else:
            variable[indexes] = T(F(xx, yy, 0))

    from itertools import permutations

    def P(x):
        return list(permutations(x))

    for d in sorted(P(["x", "y"]) + P(["time", "x", "y"])):
        write("test_2D_", d)

    for d in sorted(P(["x", "y", "z"]) + P(["time", "x", "y", "z"])):
        write("test_3D_", d)

    nc.close()


In [2]:
    def F(x, y, z):
        """A function linear in x, y, and z. Used to test our interpolation
        scheme."""
        return 10.0 + 0.01 * x + 0.02 * y + 0.03 + 0.04 * z

    # create a test file
    import tempfile
    import os

    fd, filename = tempfile.mkstemp(suffix=".nc", prefix="extract_profile_test_")
    os.close(fd)

    create_dummy_input_file(filename, F)


ModuleNotFoundError: No module named 'netCDF4'